In [3]:
#Set-Up Environment
#Import Needed Libraries
import getpass, sys
sys.path.append('/homes/sadatnfs/notebooks')
import fbd
import PyMB
from PyMB import magic
import numpy as np
import pandas as pd
import scipy as sc
import sqlalchemy as sql
import pdb
import matplotlib 
from matplotlib import pyplot as plt
import seaborn as sns
import multiprocessing as mp
from ggplot import *
from matplotlib.backends.backend_pdf import PdfPages
import os
import re
import time
idx =pd.IndexSlice

Seaborn is busted


/usr/local/codem/public_use_anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Seaborn is busted
Import Error: No module named ggplot. You will not be able to use this package's functionality, but the rest of fbd should work
Import Error: No module named pystan. You will not be able to use this package's functionality, but the rest of fbd should work
Import Error: No module named ggplot. You will not be able to use this package's functionality, but the rest of fbd should work
Seaborn is busted
PyPDF2 is busted


ImportError: No module named seaborn

In [10]:
import pandas as pd
import numpy as np

In [6]:
#Set min year 
min_year = 1980

### Load LDI

In [10]:
ldi = pd.read_csv('/share/forecasting/data/LDIpc/national_LDIpc_prepped_20161207.csv')
#ldi = pd.read_hdf(fbd.uploaders.get_directory('ldi_per_capita'), key = 'data')
ldi['LDIpc_0'] = ldi['LDIpc_1000']
ldi[['ldipc_{}'.format(i) for i in range(1000)]] = ldi[['LDIpc_{}'.format(i) for i in range(1000)]]
ldi = ldi[['ldipc_{}'.format(i) for i in range(1000)] + ['location_id', 'year_id']]
#Cap LDI draws at $60,000
ldi[['ldipc_{}'.format(i) for i in range(1000)]] = np.min(np.dstack([ldi[['ldipc_{}'.format(i) for i in range(1000)]],np.ones_like(ldi[['ldipc_{}'.format(i) for i in range(1000)]]) *60000 ]),axis=2)
ldi['ldi'] = np.log(ldi[['ldipc_{}'.format(i) for i in range(1000)]].values).mean(axis = 1)
ldi[['ldipc_{}'.format(i) for i in range(1000)]] = np.log(ldi[['ldipc_{}'.format(i) for i in range(1000)]])
ldi_cols = ['location_id', 'year_id', 'ldi']
ldi_cols = ldi_cols +['ldipc_{}'.format(i) for i in range(1000)]
ldi = ldi[ldi_cols]
ldi.head()

KeyboardInterrupt: 

In [7]:
#save LDI for 
ldi = pd.read_csv('/share/forecasting/data/LDIpc/national_LDIpc_prepped_20161207.csv')
#ldi['LDIpc_0'] = ldi['LDIpc_1000']

ldi.head()

,location_id,ihme_loc_id,year_id,age_group_id,sex_id,LDIpc_1,LDIpc_2,LDIpc_3,LDIpc_4,LDIpc_5,...,LDIpc_994,LDIpc_995,LDIpc_996,LDIpc_997,LDIpc_998,LDIpc_999,LDIpc_1000,rt_mean,rt_upper,rt_lower
0,6,CHN,1979,22,3,1015.7367,1015.7367,1015.7367,1015.7367,1015.7367,...,1015.7367,1015.7367,1015.7367,1015.7367,1015.7367,1015.7367,1015.7367,1015.736694,1015.736694,1015.736694
1,6,CHN,1980,22,3,1028.1464,1028.1464,1028.1464,1028.1464,1028.1464,...,1028.1464,1028.1464,1028.1464,1028.1464,1028.1464,1028.1464,1028.1464,1028.146362,1028.146362,1028.146362
2,6,CHN,1981,22,3,1044.3159,1044.3159,1044.3159,1044.3159,1044.3159,...,1044.3159,1044.3159,1044.3159,1044.3159,1044.3159,1044.3159,1044.3159,1044.315918,1044.315918,1044.315918
3,6,CHN,1982,22,3,1075.9132,1075.9132,1075.9130,1075.9130,1075.9130,...,1075.9130,1075.9130,1075.9132,1075.9130,1075.9130,1075.9130,1075.9130,1075.913011,1075.913208,1075.912964
4,6,CHN,1983,22,3,1114.3564,1114.3564,1114.3561,1114.3561,1114.3561,...,1114.3561,1114.3561,1114.3564,1114.3561,1114.3561,1114.3561,1114.3561,1114.356149,1114.356445,1114.356079


In [8]:
ldi_cols = ['location_id','ihme_loc_id', 'year_id', 'rt_mean']
ldi[ldi_cols].to_csv('/home/j/Project/forecasting/hiv/ref/ldi_forecasts_20161210.csv')

### Load education, Pop-Weight to All Adult

In [11]:
#%%time
pops = pd.read_csv('/ihme/forecasting/data/pop/20150101_wpp/data.csv')
#pops['pop'] = pop[['pop_{}'.format(i) for i in range(1000)]].mean(axis = 1)
#cols = ['location_id', 'sex_id', 'age_group_id', 'year_id','pop']
#edu_file=fbd.uploaders.get_directory('education')
#edu_file = edu_file.replace('.csv', '.h5')        
#edu = pd.read_hdf(edu_file, key = 'data')
edu = pd.read_csv('/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/gpr_draws.csv')
edu = edu.rename(columns={'draw{}'.format(i):'edu_forecast_draw{}'.format(i) for i in range(1000)})
edu_cols = ['location_id', 'sex_id', 'age_group_id', 'year_id']
edu_cols = edu_cols + ['edu_forecast_draw{}'.format(i) for i in range(1000)]
edu = edu[edu_cols]
edu = edu.merge(pops,how='left',on=['location_id', 'sex_id', 'age_group_id', 'year_id'])
edu.loc[:,['edu_forecast_draw{}'.format(i) for i in range(1000)]] = (edu.loc[:,['edu_forecast_draw{}'.format(i) for i in range(1000)]] * np.repeat(edu.loc[:,['pop']].values,repeats=1000,axis=1))                                                                
edu = edu.groupby(['location_id','year_id']).sum()
edu.loc[:,['edu_forecast_draw{}'.format(i) for i in range(1000)]] = (edu.loc[:,['edu_forecast_draw{}'.format(i) for i in range(1000)]] / np.repeat(edu.loc[:,['pop']].values,repeats=1000,axis=1))                                                                
edu = edu.reset_index()
edu['edu'] = edu[['edu_forecast_draw{}'.format(i) for i in range(1000)]].mean(axis = 1)
edu_cols = ['location_id', 'year_id','edu']
edu_cols = edu_cols + ['edu_forecast_draw{}'.format(i) for i in range(1000)]
edu = edu[edu_cols]
edu.head()


,location_id,year_id,edu,edu_forecast_draw0,edu_forecast_draw1,edu_forecast_draw2,edu_forecast_draw3,edu_forecast_draw4,edu_forecast_draw5,edu_forecast_draw6,...,edu_forecast_draw990,edu_forecast_draw991,edu_forecast_draw992,edu_forecast_draw993,edu_forecast_draw994,edu_forecast_draw995,edu_forecast_draw996,edu_forecast_draw997,edu_forecast_draw998,edu_forecast_draw999
0,6,1950,2.635495,2.432078,2.713488,2.444265,2.684126,2.419469,2.592997,2.535412,...,2.448978,2.456569,2.534935,2.890653,2.466241,2.551215,3.288327,2.176167,2.639432,2.735019
1,6,1951,2.716177,2.510957,2.804991,2.535165,2.754107,2.496789,2.665202,2.624748,...,2.533595,2.525153,2.629635,2.963955,2.546576,2.622928,3.380717,2.242943,2.730847,2.831496
2,6,1952,2.799702,2.595274,2.899303,2.629450,2.825219,2.577961,2.739777,2.717568,...,2.622422,2.598117,2.728182,3.039806,2.631755,2.697830,3.471224,2.312788,2.826638,2.931058
3,6,1953,2.885641,2.683937,2.996656,2.726524,2.896823,2.663117,2.816964,2.813468,...,2.714910,2.675040,2.829992,3.117165,2.721595,2.775853,3.558953,2.385391,2.925752,3.032577
4,6,1954,2.973364,2.775488,3.097141,2.825747,2.968049,2.752269,2.896925,2.911723,...,2.810392,2.755236,2.934180,3.194826,2.815741,2.856832,3.642799,2.460515,3.026658,3.134533


### Load TFR

In [26]:
#tfr = pd.read_csv(fbd.uploaders.get_directory('tfr'))
tfr = pd.read_csv('/ihme/forecasting/data/tfr/results/tfr_whole.csv')
tfr.head()

,age_group_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,sex_id,year_id
0,22,6.8066,6.8066,6.8066,6.8066,6.8066,6.8066,6.8066,6.8066,6.8066,...,6.8066,6.8066,6.8066,6.8066,6.8066,6.8066,6.8066,175.0,3,1953
1,22,6.8178,6.8178,6.8178,6.8178,6.8178,6.8178,6.8178,6.8178,6.8178,...,6.8178,6.8178,6.8178,6.8178,6.8178,6.8178,6.8178,175.0,3,1954
2,22,6.8290,6.8290,6.8290,6.8290,6.8290,6.8290,6.8290,6.8290,6.8290,...,6.8290,6.8290,6.8290,6.8290,6.8290,6.8290,6.8290,175.0,3,1955
3,22,6.8402,6.8402,6.8402,6.8402,6.8402,6.8402,6.8402,6.8402,6.8402,...,6.8402,6.8402,6.8402,6.8402,6.8402,6.8402,6.8402,175.0,3,1956
4,22,6.8514,6.8514,6.8514,6.8514,6.8514,6.8514,6.8514,6.8514,6.8514,...,6.8514,6.8514,6.8514,6.8514,6.8514,6.8514,6.8514,175.0,3,1957


In [27]:
test1 = tfr.filter(like = 'draw').mean(1)

In [28]:
#tfr = pd.read_csv(fbd.uploaders.get_directory('tfr'))
tfr = pd.read_csv('/ihme/forecasting/data/tfr/results/tfr_whole.csv')
tfr[['tfr_draw_{}'.format(i) for i in range(1000)]] = tfr[['draw_{}'.format(i) for i in range(1000)]]
tfr[['tfr_draw_{}'.format(i) for i in range(1000)]] = np.max(np.dstack([tfr[['tfr_draw_{}'.format(i) for i in range(1000)]],np.ones_like(tfr[['tfr_draw_{}'.format(i) for i in range(1000)]]) * 1 ]),axis=2)
tfr['tfr'] = tfr[['tfr_draw_{}'.format(i) for i in range(1000)]].values.mean(axis = 1)
cols = ['location_id', 'year_id', 'tfr']
cols = cols +['tfr_draw_{}'.format(i) for i in range(1000)]
tfr = tfr[cols]


In [29]:
test2 = tfr.filter(like = 'draw').mean(1)

In [32]:
(test1 == test2).all()

False

### Merge Data

In [ ]:
sds  = edu.loc[edu.year_id>=min_year].merge(ldi,how='inner',on=['location_id', 'year_id'])
sds = sds.merge(tfr,how='inner',on=['location_id', 'year_id'])


In [ ]:
#subset locations
locs = pd.read_csv('/home/j/WORK/01_covariates/02_inputs/education/update_2015/ref/IHME_GDB_2015_LOCS_6.1.15.csv')
locs = locs[['location_id','location_type_id']]
sds = sds.merge(locs,how='inner',on=['location_id'])

sds = sds[sds.location_type_id==2]

### Make SDI

In [ ]:
#Get min and max of each variable
#min_edu = np.min(sds.query('year_id < 2016 & year_id > 1979')['edu'])
#max_edu = np.max(sds.query('year_id < 2016 & year_id > 1979')['edu'])
#min_ldi = np.min(sds.query('year_id < 2016 & year_id > 1979')['ldi'])
#max_ldi = np.max(sds.query('year_id < 2016 & year_id > 1979')['ldi'])
#min_tfr = np.min(sds.query('year_id < 2016 & year_id > 1979')['tfr'])
#max_tfr = np.max(sds.query('year_id < 2016 & year_id > 1979')['tfr'])

min_edu = .595
max_edu = 15.07
min_ldi = np.log(377.16)
max_ldi = np.log(60000)
min_tfr = 1.0
max_tfr = 9.74



In [ ]:
print min_edu 
print max_edu 
print min_ldi 
print max_ldi 
print min_tfr 
print max_tfr 



In [ ]:
#Convert draws to 0 to 1 scale
sds.loc[:,['edu_forecast_draw{i}'.format(i=i) for i in range(1000)]] = ((sds.loc[:,['edu_forecast_draw{i}'.format(i=i) for i in range(1000)]] - min_edu) / (max_edu - min_edu))
sds.loc[:,['ldipc_{i}'.format(i=i) for i in range(1000)]] = ((sds.loc[:,['ldipc_{i}'.format(i=i) for i in range(1000)]] - min_ldi) / (max_ldi - min_ldi))
sds.loc[:,['tfr_draw_{i}'.format(i=i) for i in range(1000)]] = (1- ((sds.loc[:,['tfr_draw_{i}'.format(i=i) for i in range(1000)]] - min_tfr) / (max_tfr - min_tfr)))

for var in ['edu_forecast_draw','ldipc_','tfr_draw_']:
    #Set anything less than .01 to .01
    sds[['{var}{i}'.format(var=var,i=i) for i in range(1000)]] = sds[['{var}{i}'.format(var=var,i=i) for i in range(1000)]].fillna(.01)
    sds[['{var}{i}'.format(var=var,i=i) for i in range(1000)]] = np.min(np.dstack([sds[['{var}{i}'.format(var=var,i=i) for i in range(1000)]],np.ones_like(sds[['{var}{i}'.format(var=var,i=i) for i in range(1000)]]) *1 ]),axis=2)
    #Set anything more than 1 to 1
    sds[['{var}{i}'.format(var=var,i=i) for i in range(1000)]] = np.max(np.dstack([sds[['{var}{i}'.format(var=var,i=i) for i in range(1000)]],np.ones_like(sds[['{var}{i}'.format(var=var,i=i) for i in range(1000)]]) *.001 ]),axis=2)


In [ ]:
#Initilaize draws
for i in range(1000):
    sds['sds_draw{i}'.format(i=i)] = 0.

In [ ]:
#Make SDS Draws
sds[['sds_draw{i}'.format(i=i) for i in range(1000) ]] =  \
    ((sds.loc[:,['edu_forecast_draw{i}'.format(i=i) for i in range(1000)]].values * \
    sds.loc[:,['ldipc_{i}'.format(i=i) for i in range(1000)]].values * \
     sds.loc[:,['tfr_draw_{i}'.format(i=i) for i in range(1000)]].values)**(.333333333333333))


In [ ]:
#Set anything less than .01 to .01
sds[['sds_draw{i}'.format(i=i) for i in range(1000)]] = sds[['sds_draw{i}'.format(i=i) for i in range(1000)]].fillna(.01)
sds[['sds_draw{}'.format(i) for i in range(1000)]] = np.min(np.dstack([sds[['sds_draw{}'.format(i) for i in range(1000)]],np.ones_like(sds[['sds_draw{}'.format(i) for i in range(1000)]]) *1 ]),axis=2)

#Set anything more than 1 to 1
sds[['sds_draw{}'.format(i) for i in range(1000)]] = np.max(np.dstack([sds[['sds_draw{}'.format(i) for i in range(1000)]],np.ones_like(sds[['sds_draw{}'.format(i) for i in range(1000)]]) *.001 ]),axis=2)


In [ ]:
#only keep min years onwards


### Save SDI

In [ ]:
#make directory if needed
date = (time.strftime("%Y-%m-%d"))
print date
directory = '/ihme/forecasting/data/covs/rt/SDI/3/{date}/forecast/'.format(date = date)
if not os.path.exists(directory):
    os.makedirs(directory)
output_path = '/ihme/forecasting/data/covs/rt/SDI/3/{date}/forecast/SDI.hdf'.format(date = date)
print output_path

In [ ]:
#Save SDS draws
sds['age_group_id'] = 22
sds['sex_id'] = 3
sds.loc[:,['location_id', 'sex_id', 'age_group_id', 'year_id'] +\
            ['sds_draw{i}'.format(i=i) for i in range(1000)]].to_hdf(output_path,key='data',format='table', data_columns=['location_id', 'age_group_id', 'year_id', 'sex_id'])


In [7]:
#Upload draws
directory = '/ihme/forecasting/data/covs/rt/SDI/3/2016-12-08/forecast/SDI.hdf'

fbd.uploaders.prep_data(directory, col_type = 'rt',
                                     entity = 'sdi', outpath = '/ihme/forecasting/data/covariates/sds_draws_temp_upload_5.csv',
                                     notes = 'SDI 12/8/16',gbd_inputs_round=2015)

(       model_id  entity_id  location_id  age_group_id  sex_id  year_id  \
 0            27          1            6            22       3     1980   
 1            27          1            6            22       3     1981   
 2            27          1            6            22       3     1982   
 3            27          1            6            22       3     1983   
 4            27          1            6            22       3     1984   
 5            27          1            6            22       3     1985   
 6            27          1            6            22       3     1986   
 7            27          1            6            22       3     1987   
 8            27          1            6            22       3     1988   
 9            27          1            6            22       3     1989   
 10           27          1            6            22       3     1990   
 11           27          1            6            22       3     1991   
 12           27         

In [ ]:
#Calculate means and CIs
#means
sds['edu_mean'] = sds[['edu_forecast_draw{}'.format(i) for i in range(1000)]].mean(axis = 1)
sds['ldi_mean'] = sds[['ldipc_{}'.format(i) for i in range(1000)]].mean(axis = 1)
sds['sds_mean'] = sds[['sds_draw{}'.format(i) for i in range(1000)]].mean(axis = 1)
sds['tfr_mean'] = sds[['tfr_draw_{}'.format(i) for i in range(1000)]].mean(axis = 1)
#uppers
sds['edu_upper'] = sds[['edu_forecast_draw{}'.format(i) for i in range(1000)]].quantile(q=.975,axis=1)
sds['ldi_upper'] = sds[['ldipc_{}'.format(i) for i in range(1000)]].quantile(q=.975,axis=1)
sds['sds_upper'] = sds[['sds_draw{}'.format(i) for i in range(1000)]].quantile(q=.975,axis=1)
sds['tfr_upper'] = sds[['tfr_draw_{}'.format(i) for i in range(1000)]].quantile(q=.975,axis=1)

#lowers
sds['edu_lower'] = sds[['edu_forecast_draw{}'.format(i) for i in range(1000)]].quantile(q=.025,axis=1)
sds['ldi_lower'] = sds[['ldipc_{}'.format(i) for i in range(1000)]].quantile(q=.025,axis=1)
sds['sds_lower'] = sds[['sds_draw{}'.format(i) for i in range(1000)]].quantile(q=.025,axis=1)
sds['tfr_lower'] = sds[['tfr_draw_{}'.format(i) for i in range(1000)]].quantile(q=.025,axis=1)

sds.head()

In [ ]:
sds['point_estimate'] = sds['sds_mean']
sds['lower_bound'] = sds['sds_lower']
sds['upper_bound'] = sds['sds_upper']
sds['response_var_T'] = 0
sds['oos_point_estimate'] = np.nan
sds['oos_lower_bound'] = np.nan
sds['oos_upper_bound'] = np.nan

In [ ]:
#Save summaries for viz tool
summary = sds[['location_id','age_group_id','year_id','sex_id','response_var_T','point_estimate','lower_bound','upper_bound','oos_point_estimate','oos_lower_bound','oos_upper_bound']]
summary.to_hdf(output_path,method='a',key='summary',format='table', data_columns=['location_id', 'age_group_id', 'year_id', 'sex_id'])

In [ ]:
#locs = fbd.downloaders.super_region_map()
#locs[['location_name']] = locs[['db_name']]
#loc_names = locs[['location_name','location_id']]
#loc_names.head()
#sds = sds.merge(loc_names,how='left',on='location_id')
#sds.head()

In [ ]:
#Convert LDI and TFR back to normal Space
#sds.tfr = np.exp(sds.tfr)
#sds.ldi_mean = np.exp(sds.ldi_mean)
#sds.ldi_upper = np.exp(sds.ldi_upper)
#sds.ldi_lower = np.exp(sds.ldi_lower)

In [ ]:
#Save all data for viz tool
sds.loc[:,['location_id', 'year_id','edu_mean','sds_mean','ldi_mean','edu_upper','sds_upper','ldi_upper',
           'edu_lower','ldi_lower','sds_lower','tfr_mean','tfr_lower','tfr_upper']].to_hdf(output_path,method='a',key='custom_viz')